# Iteration 4: 

## Plan English Summary

This is a non-terminating simulation.  The next step is to implement a warm-up period parameter for the model.  This is user settable.




## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## Show query 

In [2]:
display(Markdown("inputs/iteration_4.md"))

modify the code as follows

the model should include a user settable warm up period. This defaults to 1 month.

model run length should be renamed results collection period and have the default value of 12 months.  

The total run length of the model is the warm up period plus the results collection period.

the Experiment class should also accept a parameter for the warm up period

Show all of the model code: this includes all patient generator functions in the CCModel class and all code in the Experiment class.

## Date and time of input to LLM

> 1840 29th November 2023 

## Observations on the LLM Output


* The warm-up period has been added to `Experiment` as a parameter.
* `run_length` has been correctly renamed to `results_collection_period`
* The model is run for the `results_collection_period` + `warm_up_period`
* At the moment the warm-up period is not used anywhere else in the model.

## LLM Output 1:

In this modified code, the Experiment class accepts a parameter for the warm up period, and the run method of the CCUModel class refers to self.env instead of the variable env. The model run length is renamed results collection period and has the default value of 12 months. The total run length of the model is the warm up period plus the results collection period

In [3]:
import simpy
import numpy as np

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)


In [4]:
# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

Patient 1 arrived from Accident and Emergency at time 6.074972656981616
Patient 1 admitted to critical care bed at time 6.074972656981616
Patient 2 arrived from Emergency surgery at time 8.779921832940607
Patient 2 admitted to critical care bed at time 8.779921832940607
Patient 3 arrived from other hospitals at time 13.135406523346122
Patient 3 admitted to critical care bed at time 13.135406523346122
Patient 4 arrived from other hospitals at time 14.596999695145312
Patient 4 admitted to critical care bed at time 14.596999695145312
Elective surgery patient 5 arrived at time 18.312755991170782
Elective surgery patient 5 admitted to critical care bed at time 18.312755991170782
Patient 6 arrived from Accident and Emergency at time 22.33856307160029
Patient 6 admitted to critical care bed at time 22.33856307160029
Patient 7 arrived from Emergency surgery at time 24.39724544588472
Patient 7 admitted to critical care bed at time 24.39724544588472
Patient 8 arrived from Accident and Emergency 

## Modifications to LLM code to test for correct model outputs.

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class.

From iteration 0

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than 240 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.


In [5]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [6]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [7]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [8]:
extreme_value_test_1()

Patient 1 arrived from Accident and Emergency at time 8.218312305042499
Patient 1 admitted to critical care bed at time 8.218312305042499
Patient 2 arrived from Accident and Emergency at time 35.32526685428359
Patient 2 admitted to critical care bed at time 35.32526685428359
Patient 3 arrived from Accident and Emergency at time 87.37982265255921
Patient 3 admitted to critical care bed at time 87.37982265255921
Patient 4 arrived from Accident and Emergency at time 93.32139884910228
Patient 4 admitted to critical care bed at time 93.32139884910228
Patient 4 discharged from critical care bed at time 96.13397845611746
Patient 5 arrived from Accident and Emergency at time 100.0445092765828
Patient 5 admitted to critical care bed at time 100.0445092765828
Critical care bed is available for next patient at time 101.13397845611746
Patient 5 discharged from critical care bed at time 125.32928981859331
Critical care bed is available for next patient at time 130.3292898185933
Patient 5 discharged

### Test 2: Extreme value test 2

PASS

In [9]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [10]:
extreme_value_test_2()

Elective surgery patient 1 arrived at time 15.67608078245054
Elective surgery patient 1 admitted to critical care bed at time 15.67608078245054
Elective surgery patient 2 arrived at time 34.98997380709969
Elective surgery patient 2 admitted to critical care bed at time 34.98997380709969
Elective surgery patient 3 arrived at time 50.00289129789258
Elective surgery patient 3 admitted to critical care bed at time 50.00289129789258
Elective surgery patient 4 arrived at time 65.13036443258112
Elective surgery patient 4 admitted to critical care bed at time 65.13036443258112
Elective surgery patient 4 discharged from critical care bed at time 70.49754464724995
Critical care bed is available for next patient at time 75.49754464724995
Elective surgery patient 5 arrived at time 79.9120690252521
Elective surgery patient 5 admitted to critical care bed at time 79.9120690252521
Elective surgery patient 6 arrived at time 96.55065799845922
Elective surgery patient 6 admitted to critical care bed at 

### Test 3: Different run length

PASS

In [11]:
def test_run_length(new_run_length):
    '''
    Different run length

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_run_length)
    model = run_test(experiment)
    return model.env.now

In [12]:
test_run_length(10)

Patient 1 arrived from Accident and Emergency at time 7.109453413489351
Patient 1 admitted to critical care bed at time 7.109453413489351
Patient 2 arrived from the X-Ray department at time 12.29912075228718
Patient 2 admitted to critical care bed at time 12.29912075228718
Patient 3 arrived from the Wards at time 12.820786462881433
Patient 3 admitted to critical care bed at time 12.820786462881433
Patient 4 arrived from Emergency surgery at time 15.660709611384618
Patient 4 admitted to critical care bed at time 15.660709611384618
Elective surgery patient 5 arrived at time 18.551403087836047
Elective surgery patient 5 admitted to critical care bed at time 18.551403087836047
Patient 6 arrived from Accident and Emergency at time 20.559341726813443
Patient 6 admitted to critical care bed at time 20.559341726813443
Patient 6 discharged from critical care bed at time 33.347628370179834
Elective surgery patient 7 arrived at time 38.2652682525479
Elective surgery patient 7 admitted to critical

730

### Test 4: Extreme value test 3

PASS.

In [13]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)

In [14]:
extreme_value_test_3()

Patient 1 arrived from Emergency surgery at time 3.1866980376721914
Patient 1 admitted to critical care bed at time 3.1866980376721914
Patient 2 arrived from other hospitals at time 3.465057053485271
Patient 3 arrived from the Wards at time 6.890844582784977
Elective surgery patient 4 arrived at time 16.6318229921661
Elective surgery for patient 4 cancelled due to no available critical care beds at time 16.6318229921661
Patient 5 arrived from the Wards at time 18.94266588160805
Patient 6 arrived from Emergency surgery at time 19.240380154601098
Patient 7 arrived from the Wards at time 25.753939388940296
Elective surgery patient 8 arrived at time 35.441083576827154
Elective surgery for patient 8 cancelled due to no available critical care beds at time 35.441083576827154
Patient 9 arrived from the Wards at time 50.8218774872212
Elective surgery patient 10 arrived at time 50.95512799816985
Elective surgery for patient 10 cancelled due to no available critical care beds at time 50.95512799

### Test 5: Extreme value test 4

PASS

In [15]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [16]:
extreme_value_test_4()

Patient 1 arrived from the Wards at time 4.007841908173693
Patient 1 admitted to critical care bed at time 4.007841908173693
Patient 2 arrived from Accident and Emergency at time 9.891556818827103
Patient 2 admitted to critical care bed at time 9.891556818827103
Elective surgery patient 3 arrived at time 15.433109343897591
Elective surgery patient 3 admitted to critical care bed at time 15.433109343897591
Patient 4 arrived from Accident and Emergency at time 27.301673769082804
Patient 4 admitted to critical care bed at time 27.301673769082804
Elective surgery patient 5 arrived at time 33.09778394197086
Elective surgery patient 5 admitted to critical care bed at time 33.09778394197086
Patient 6 arrived from Emergency surgery at time 34.17392190840497
Patient 6 admitted to critical care bed at time 34.17392190840497
Patient 7 arrived from the Wards at time 38.25640816699311
Patient 7 admitted to critical care bed at time 38.25640816699311
Patient 8 arrived from other hospitals at time 48